# Building Fast Queries On a CSV

In this project, we are given a csv containing laptop data:

**ID:** A unique identifier for the laptop.  
**Company:** The name of the company that produces the laptop.  
**Product:** The name of the laptop.  
**TypeName:** The type of laptop.  
**Inches:** The size of the screen in inches.  
**ScreenResolution:** The resolution of the screen.  
**CPU**: The laptop CPU.  
**RAM**: The amount of RAM in the laptop.  
**Memory**: The size of the hard drive.  
**GPU**: The graphics card name.  
**OpSys**: The name of the operating system.  
**Weight**: The laptop weight.  
**Price**: The price of the laptop.  

We want to build a way to answer certain questions about our laptop inventory in an efficient manner.

## Objectives
1. Analyze the time and space complexity of an algorithm.
2. Preprocesse data to significantly speed-up an algorithm.
3. Sort data and efficiently search sorted data.

## The Dataset

In [1]:
import csv
import time
import random

In [2]:
with open("laptops.csv") as file:
    file_contents = list(csv.reader(file))
    header = file_contents[0]
    rows = file_contents[1:]

print("Header is:", header)
print("\n")
print("First five rows:", rows[:5])

Header is: ['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price']


First five rows: [['6571244', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 2.3GHz', '8GB', '128GB SSD', 'Intel Iris Plus Graphics 640', 'macOS', '1.37kg', '1339'], ['7287764', 'Apple', 'Macbook Air', 'Ultrabook', '13.3', '1440x900', 'Intel Core i5 1.8GHz', '8GB', '128GB Flash Storage', 'Intel HD Graphics 6000', 'macOS', '1.34kg', '898'], ['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', '575'], ['9722156', 'Apple', 'MacBook Pro', 'Ultrabook', '15.4', 'IPS Panel Retina Display 2880x1800', 'Intel Core i7 2.7GHz', '16GB', '512GB SSD', 'AMD Radeon Pro 455', 'macOS', '1.83kg', '2537'], ['8550527', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel C

## Inventory Class

In [3]:
class Inventory():
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            file_contents = list(csv.reader(file))
            
        self.header = file_contents[0]
        self.rows = file_contents[1:]
        
        for row in self.rows:
            row[-1] = int(row[-1])
            
# Testing Inventory Class
laptop_inventory = Inventory('laptops.csv')
print("Laptop Inventory Header:", laptop_inventory.header)
print("\n")
print("Number of rows in Inventory:", len(laptop_inventory.rows))

Laptop Inventory Header: ['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price']


Number of rows in Inventory: 1303


## Finding a Laptop From the Id

The first thing that we will implement is a way to look up a laptop from a given identifier. In this way, when a customer comes to our store with a purchase slip, we can quickly identify the laptop to which it corresponds.

In [4]:
class Inventory():
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            file_contents = list(csv.reader(file))
            
        self.header = file_contents[0]
        self.rows = file_contents[1:]
        
        for row in self.rows:
            row[-1] = int(row[-1])
    
    def get_laptop_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None
    
# Testing get_laptop_id function
laptop_inventory = Inventory('laptops.csv')
print(laptop_inventory.get_laptop_id('3362737')) # This should find a laptop
print(laptop_inventory.get_laptop_id('3362736')) # This should not find a laptop

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]
None


## Improving ID Lookups

The idea for this optimization is to preprocess the data into a dictionary where the keys are the IDs and the values are the rows. This will give us constant time lookups in cases where we want to find a row(s) that has a certain ID.

We will continue to copy and paste each iteration of the Inventory class so we can keep track of our changes.

In [5]:
class Inventory():
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            file_contents = list(csv.reader(file))
            
        self.header = file_contents[0]
        self.rows = file_contents[1:]
        self.id_to_row = {}
        
        for row in self.rows:
            row[-1] = int(row[-1])
            self.id_to_row[row[0]] = row
            
    def get_laptop_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
# Testing get_laptop_id function
laptop_inventory = Inventory('laptops.csv')
print(laptop_inventory.get_laptop_from_id_fast('3362737')) # This should find a laptop
print(laptop_inventory.get_laptop_from_id_fast('3362736')) # This should not find a laptop

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]
None


## Comparing the Performance

Let's compare the performance of both methods. We will generate some random IDs, then use each method to lookup these same IDs.

In [6]:
ids = [str(random.randint(1000000, 9999999)) for _ in range(10000)]
laptop_inventory = Inventory('laptops.csv')
total_time_no_dict = 0
for laptop_id in ids:
    start = time.time()
    laptop_inventory.get_laptop_id(laptop_id)
    end = time.time()
    total_time_no_dict += end - start
    
total_time_dict = 0
for laptop_id in ids:
    start = time.time()
    laptop_inventory.get_laptop_from_id_fast(laptop_id)
    end = time.time()
    total_time_dict += end - start
    
print("Total time elapsed when using get_laptop_id:", total_time_no_dict)
print("Total time elapsed when using get_laptop_from_id_fast:", total_time_dict)

Total time elapsed when using get_laptop_id: 0.7025125026702881
Total time elapsed when using get_laptop_from_id_fast: 0.0028009414672851562


As we can see above, get_laptop_from_id_fast, which uses the data preprocessed into a dictionary, is about 271 times faster!

## Two Laptop Promotion

Let's say that there is a promotional gift card that allows a customer to buy up to 2 laptops. The catch to this gift card is that it can only be used once. Even if the customer uses the gift card and there is money left over, the customer can no longer use the gift card.

In order to make the customer feel like they are not being cheated, we want to make sure there is either one laptop that costs D (the amount of money on the gift card) dollars, or 2 laptops that add up to exactly D dollars.

We will implement this function now.

In [7]:
class Inventory():
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            file_contents = list(csv.reader(file))
            
        self.header = file_contents[0]
        self.rows = file_contents[1:]
        self.id_to_row = {}
        
        for row in self.rows:
            row[-1] = int(row[-1])
            self.id_to_row[row[0]] = row
            
    def get_laptop_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            if row[-1] == dollars:
                return True
        
        for row_outside in self.rows:
            for row_inside in self.rows:
                if row_outside[-1] + row_inside[-1] == dollars:
                    return True
        
        return False
    
laptop_inventory = Inventory('laptops.csv')
print(laptop_inventory.check_promotion_dollars(1000)) # This should be True
print(laptop_inventory.check_promotion_dollars(442)) # This should be False

True
False


## Optimizing Laptop Promotion

We've learned that preprocessing data can help run these types of queries faster. We only care about whether or not there is a solution, so let's store the laptop prices in a set. Set's provide us with constant time lookup.

We can use the following logic to determine whether or not a pair of laptops add up to the gift card dollar amount:
Laptop_1 + Laptop_2 = Total
Laptop_1 = Total - Laptop_2
OR
Laptop_2 = Total - Laptop_1

We can subtract the giftcard amount (the provided dollar amount in our check_promotion function) with an existing value in the set and see if the difference is in the set.

In [8]:
class Inventory():
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            file_contents = list(csv.reader(file))
            
        self.header = file_contents[0]
        self.rows = file_contents[1:]
        self.id_to_row = {}
        self.prices = set()
        
        for row in self.rows:
            row[-1] = int(row[-1])
            self.id_to_row[row[0]] = row
            self.prices.add(row[-1])
            
    # Runtime: O(N) where N is length of self.rows
    def get_laptop_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None
    
    # Runtime: O(1) because dictionaries give us constant lookup time when provided with a key
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
    # Runtime: O(N^2) where N is length of self.rows
    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            if row[-1] == dollars:
                return True
        
        for row_outside in self.rows:
            for row_inside in self.rows:
                if row_outside[-1] + row_inside[-1] == dollars:
                    return True
        
        return False
    
    # Runtime: O(N) where N is length of self.prices
    def check_promotion_dollars_fast(self, dollars):
        if dollars in self.prices:
            return True
        
        for price in self.prices:
            if (dollars - price) in self.prices:
                return True
        
        return False
    
laptop_inventory = Inventory('laptops.csv')
print(laptop_inventory.check_promotion_dollars_fast(1000)) # This should be True
print(laptop_inventory.check_promotion_dollars_fast(442)) # This should be False

True
False


## Comparing Performance of Laptop Promotion Function

In [9]:
prices = [random.randint(100, 5000) for _ in range(100)]
laptop_inventory = Inventory('laptops.csv')
total_time_no_set = 0
for price in prices:
    start = time.time()
    laptop_inventory.check_promotion_dollars(price)
    end = time.time()
    total_time_no_set += end - start

total_time_set = 0
for price in prices:
    start = time.time()
    laptop_inventory.check_promotion_dollars_fast(price)
    end = time.time()
    total_time_set += end - start
    
print("Total calculation time when using no set:", total_time_no_set)
print("Total calculation time when using set:", total_time_set)

Total calculation time when using no set: 0.602515697479248
Total calculation time when using set: 0.0003750324249267578


As we can see, preprocessing the prices into a set makes finding matching promotions over 2000 times faster!

## Finding Laptops Within a Budget

Let's write a method that helps a customer find all laptops that fall within their budget. Given a budget of D dollars, find all laptops whose price is at most D.

We can do this efficiently by sorting all laptops by price, then using binary search to identify the first laptop in the sorted list with a price larger than D. From there, any laptop with index less than the one we found will fall within the customer's budget.

In [10]:
class Inventory():
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            file_contents = list(csv.reader(file))
            
        self.header = file_contents[0]
        self.rows = file_contents[1:]
        self.id_to_row = {}
        self.prices = set()
        
        for row in self.rows:
            row[-1] = int(row[-1])
            self.id_to_row[row[0]] = row
            self.prices.add(row[-1])
        
        def row_price(row):
            return row[-1]
            
        self.rows_by_price = sorted(self.rows, key=row_price)
    
    # Runtime: O(N) where N is length of self.rows
    def get_laptop_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None
    
    # Runtime: O(1) because dictionaries give us constant lookup time when provided with a key
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
    # Runtime: O(N^2) where N is length of self.rows
    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            if row[-1] == dollars:
                return True
        
        for row_outside in self.rows:
            for row_inside in self.rows:
                if row_outside[-1] + row_inside[-1] == dollars:
                    return True
        
        return False
    
    # Runtime: O(N) where N is length of self.prices
    def check_promotion_dollars_fast(self, dollars):
        if dollars in self.prices:
            return True
        
        for price in self.prices:
            if (dollars - price) in self.prices:
                return True
        
        return False
    
    def find_first_laptop_more_expensive(self, price):
        low = 0
        high = len(self.rows_by_price) - 1
        
        if self.rows_by_price[-1][-1] <= price:
            return -1
        
        if self.rows_by_price[0][-1] > price:
            return 0
        
        while low < high:
            current_index = (low + high) // 2
            current_row = self.rows_by_price[current_index]
            current_price = current_row[-1]
            if current_price <= price:
                low = current_index + 1
            elif current_price > price:
                high = current_index
        
        return high
    
laptop_inventory = Inventory('laptops.csv')
print(laptop_inventory.find_first_laptop_more_expensive(1000)) # Should return index 683
print(laptop_inventory.find_first_laptop_more_expensive(10000)) # Should return index -1

683
-1


The above indicates that if the customer's budget is 1,000 dollars, there are 683 laptops the customer can afford. If the customer's budget is 10,000 dollars, there are no laptops the customer cannot afford.

## Further Queries

Now let's try and answer the following queries:

1. Imagine that we extend our budget query to take as input a range of prices, min_price and max_price, rather than a single price. Write a query that finds all laptops whose price is in the given range.

2. Sometimes, a customer wants a laptop with some characteristics such as, for instance, 8GB or RAM and a 256GB hard drive. It would be interesting for those customers to provide a way to find the cheapest laptop that matches the desired characteristics. For simplicity, focus only on the amount of RAM and hard drive capacity. You might need to convert those values to integers rather than using strings.

For the first query, we can use find_first_laptop_more_expensive on max_price, and we can write a similar find_first_laptop_less_expensive helper function and use it on min_price.

In [17]:
class Inventory():
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            file_contents = list(csv.reader(file))
            
        self.header = file_contents[0]
        self.rows = file_contents[1:]
        self.id_to_row = {}
        self.prices = set()
        
        for row in self.rows:
            row[-1] = int(row[-1])
            self.id_to_row[row[0]] = row
            self.prices.add(row[-1])
        
        def row_price(row):
            return row[-1]
            
        self.rows_by_price = sorted(self.rows, key=row_price)
    
    # Runtime: O(N) where N is length of self.rows
    def get_laptop_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None
    
    # Runtime: O(1) because dictionaries give us constant lookup time when provided with a key
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
    # Runtime: O(N^2) where N is length of self.rows
    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            if row[-1] == dollars:
                return True
        
        for row_outside in self.rows:
            for row_inside in self.rows:
                if row_outside[-1] + row_inside[-1] == dollars:
                    return True
        
        return False
    
    # Runtime: O(N) where N is length of self.prices
    def check_promotion_dollars_fast(self, dollars):
        if dollars in self.prices:
            return True
        
        for price in self.prices:
            if (dollars - price) in self.prices:
                return True
        
        return False
    
    def find_first_laptop_more_expensive(self, price):
        low = 0
        high = len(self.rows_by_price) - 1
        
        if self.rows_by_price[-1][-1] <= price:
            return -1
        
        if self.rows_by_price[0][-1] > price:
            return 0
        
        while low < high:
            current_index = (low + high) // 2
            current_row = self.rows_by_price[current_index]
            current_price = current_row[-1]
            if current_price <= price:
                low = current_index + 1
            elif current_price > price:
                high = current_index
        
        return high
    
    def find_first_laptop_less_expensive(self, price):
        low = 0
        high = len(self.rows_by_price) - 1
        
        if self.rows_by_price[0][-1] >= price:
            return -1
        
        if self.rows_by_price[-1][-1] < price:
            return len(self.rows_by_price) - 1
        
        while low < high:
            current_index = (low + high) // 2
            current_row = self.rows_by_price[current_index]
            current_price = current_row[-1]
            if current_price < price:
                low = current_index
            elif current_price >= price:
                high = current_index - 1
                
        return high
    
    # Returns a tuple (low, high)
    # low corresponds to the index of the first laptop that is cheaper than min_price.
    # high corresponds to the index of the first laptop that is more expensive than max_price.
    def find_laptops_within_price_range(self, min_price, max_price):
        low = self.find_first_laptop_less_expensive(min_price)
        high = self.find_first_laptop_more_expensive(max_price)
        return low, high
        
laptop_inventory = Inventory('laptops.csv')
low_index, high_index = laptop_inventory.find_laptops_within_price_range(500, 1500)
print("Index of first laptop cheaper than $500:", low_index)
print("Index of first laptop more expensive than $1500:", high_index)
print("Checking the laptop prices around cheapest laptop index:")
print("Index " + str(low_index - 1) + ": " + str(laptop_inventory.rows_by_price[low_index - 1][-1]))
print("Index " + str(low_index) + ": " + str(laptop_inventory.rows_by_price[low_index][-1]))
print("Index " + str(low_index + 1) + ": " + str(laptop_inventory.rows_by_price[low_index + 1][-1]))
print("Checking the laptop prices around most expensive laptop index:")
print("Index " + str(high_index - 1) + ": " + str(laptop_inventory.rows_by_price[high_index - 1][-1]))
print("Index " + str(high_index) + ": " + str(laptop_inventory.rows_by_price[high_index][-1]))
print("Index " + str(high_index + 1) + ": " + str(laptop_inventory.rows_by_price[high_index + 1][-1]))

Index of first laptop cheaper than $500: 244
Index of first laptop more expensive than $1500: 999
Checking the laptop prices around cheapest laptop index:
Index 243: 499
Index 244: 499
Index 245: 500
Checking the laptop prices around most expensive laptop index:
Index 998: 1500
Index 999: 1504
Index 1000: 1510


For the 2nd query, we could filter laptop data by the provided memory and hard drive amounts and return the laptop at index 0 (or None if no laptops match the description). We can use the existing sorted laptop list so that the laptop at index 0 will be the cheapest one.

In [20]:
class Inventory():
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            file_contents = list(csv.reader(file))
            
        self.header = file_contents[0]
        self.rows = file_contents[1:]
        self.id_to_row = {}
        self.prices = set()
        
        for row in self.rows:
            row[-1] = int(row[-1])
            self.id_to_row[row[0]] = row
            self.prices.add(row[-1])
        
        def row_price(row):
            return row[-1]
            
        self.rows_by_price = sorted(self.rows, key=row_price)
    
    # Runtime: O(N) where N is length of self.rows
    def get_laptop_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None
    
    # Runtime: O(1) because dictionaries give us constant lookup time when provided with a key
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
    # Runtime: O(N^2) where N is length of self.rows
    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            if row[-1] == dollars:
                return True
        
        for row_outside in self.rows:
            for row_inside in self.rows:
                if row_outside[-1] + row_inside[-1] == dollars:
                    return True
        
        return False
    
    # Runtime: O(N) where N is length of self.prices
    def check_promotion_dollars_fast(self, dollars):
        if dollars in self.prices:
            return True
        
        for price in self.prices:
            if (dollars - price) in self.prices:
                return True
        
        return False
    
    def find_first_laptop_more_expensive(self, price):
        low = 0
        high = len(self.rows_by_price) - 1
        
        if self.rows_by_price[-1][-1] <= price:
            return -1
        
        if self.rows_by_price[0][-1] > price:
            return 0
        
        while low < high:
            current_index = (low + high) // 2
            current_row = self.rows_by_price[current_index]
            current_price = current_row[-1]
            if current_price <= price:
                low = current_index + 1
            elif current_price > price:
                high = current_index
        
        return high
    
    def find_first_laptop_less_expensive(self, price):
        low = 0
        high = len(self.rows_by_price) - 1
        
        if self.rows_by_price[0][-1] >= price:
            return -1
        
        if self.rows_by_price[-1][-1] < price:
            return len(self.rows_by_price) - 1
        
        while low < high:
            current_index = (low + high) // 2
            current_row = self.rows_by_price[current_index]
            current_price = current_row[-1]
            if current_price < price:
                low = current_index
            elif current_price >= price:
                high = current_index - 1
                
        return high
    
    # Returns a tuple (low, high)
    # low corresponds to the index of the first laptop that is cheaper than min_price.
    # high corresponds to the index of the first laptop that is more expensive than max_price.
    def find_laptops_within_price_range(self, min_price, max_price):
        low = self.find_first_laptop_less_expensive(min_price)
        high = self.find_first_laptop_more_expensive(max_price)
        return low, high
    
    # We can use the sorted laptop list so we don't have to re-sort by price.
    def find_cheapest_laptop_ram_memory(self, ram, memory):
        filtered_laptops = []
        for row in self.rows_by_price:
            current_ram = row[7]
            current_memory = row[8]
            if (current_ram == ram) and (current_memory == memory):
                filtered_laptops.append(row)
        if len(filtered_laptops) > 0:
            return filtered_laptops[0]
        return None
        
laptop_inventory = Inventory('laptops.csv')
print(laptop_inventory.find_cheapest_laptop_ram_memory('8GB', '256GB SSD'))

['2618101', 'Acer', 'ES1-523-84K7 (A8-7410/8GB/256GB/FHD/W10)', 'Notebook', '15.6', 'Full HD 1920x1080', 'AMD A8-Series 7410 2.2GHz', '8GB', '256GB SSD', 'AMD Radeon R5', 'Windows 10', '2.23kg', 469]


I decided to use strings as inputs for memory and hard drive so the user can specify GB, TB, and whether or not the hard drive is SSD or HDD.